In [9]:
import os
import pickle
import sqlite3
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [14]:
# Connect to database
path = os.path.join('..', 'Database', 'news.db')
conn = sqlite3.connect(path)
cursor = conn.cursor()

In [7]:
# Load TF-IDF features
cursor.execute("SELECT data FROM features WHERE type = 'tfidf'")
X_tfidf_compressed = cursor.fetchone()[0]
X_tfidf = pickle.loads(X_tfidf_compressed)
print("Features loaded successfully!")

Features loaded successfully!


In [ ]:
# Load labels from the original dataset
conn = sqlite3.connect("fake_news.db")
df = pd.read_sql("SELECT id, label FROM news_cleaned", conn)
conn.close()

# Assuming X_tfidf is the extracted features
y = df["label"].values  # Labels (0: Fake, 1: Real)

# Split data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

print("✅ Data split into training and testing sets!")
print("Train size:", X_train.shape, "Test size:", X_test.shape)

In [59]:
# Load labels from the original dataset
df= pd.read_sql("SELECT id, label FROM cleanedText", conn) 
df['label'] = df['label'].apply(lambda x: 1 if x == 'real' else (0 if x == 'fake' else None))

In [60]:
# Assuming X_tfidf is the extracted features
y = df['label'].values # Labels (0: Fake, 1: Real)

In [61]:
# Split data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=1311)
print("Data split into training and testing sets!")
print("Train size: ", X_train.shape, "Test size: ", X_test.shape)

Data split into training and testing sets!
Train size:  (35918, 5000) Test size:  (8980, 5000)
